## Google Assistant with Blinky Light Action

This is the code you used earlier to run the Google Assistant on the AIY Voice Kit. 

In [1]:
import argparse
import locale
import logging
import signal
import sys
import time

from aiy.assistant.grpc import AssistantServiceClientWithLed
from aiy.assistant import auth_helpers, device_helpers, device_handler_helpers
from aiy.board import Board
from aiy.leds import Leds, Color

Things we need to blink the LED

In [2]:
from gpiozero import LED
from aiy.pins import PIN_A

led = LED(PIN_A)

#### Setup

Here we set the volume (0-100), language and create a logger object allowing all the different parts of the code to write messages to a log - or here to the screen.

In [3]:
logger = logging.getLogger(__name__)
language, _ = locale.getdefaultlocale()
volume = 10

logging.basicConfig(level=logging.DEBUG)
signal.signal(signal.SIGTERM, lambda signum, frame: sys.exit(0))

credentials = auth_helpers.get_assistant_credentials()
device_model_id, device_id = device_helpers.get_ids_for_service(credentials)
device_handler = device_handler_helpers.DeviceRequestHandler(device_id)

logger.info('device_model_id: %s', device_model_id)
logger.info('device_id: %s', device_id)

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
INFO:__main__:device_model_id: blinkagain-84b8d-blinkagain-gkpqln
INFO:__main__:device_id: credible-acre-346021-voice-kit-c4a5e41d-81b9-4282-b312-88075b4aa031


#### Custom handlers

Once we have configured curom traits (like turning the servo), we need to add the code that handles those requests here.

In [6]:
#------------------------------------------------------------------------------
# Add custom device handlers here
#------------------------------------------------------------------------------

@device_handler.command('com.example.commands.BlinkLight')

def blink(speed, number):
    logging.info('Blinking device %s times.' % number)
    delay = 0.5
    if speed == "SLOWLY":
        delay = 1
    elif speed == "QUICKLY":
        delay = 0.2

    for i in range(int(number)):
        logging.info('Device is blinking now.')
        led.on()
        time.sleep(delay)
        led.off()
        time.sleep(delay)

#------------------------------------------------------------------------------    

#### Assistant loop

Here is the main part. Basically it waits for the user to press the button and then calls the assistent.conversation() routine to deal with one convsersation. Once it send it goes back waiting for someone to press the button and so on.

In [7]:
with Board() as board:
    assistant = AssistantServiceClientWithLed(board=board,
                                              volume_percentage=volume,
                                              language_code=language,
                                              credentials=credentials,
                                              device_model_id=device_model_id,
                                              device_id=device_id,
                                              device_handler=device_handler)

    try:
        while True:
            logging.info('Press button to start conversation...')
            board.button.wait_for_press()
            logging.info('Conversation started!')
            assistant.conversation()
    except KeyboardInterrupt:
        print("Bye")

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
INFO:aiy.assistant.grpc:Connecting to embeddedassistant.googleapis.com
INFO:root:Press button to start conversation...
INFO:root:Conversation started!
INFO:aiy.assistant.grpc:Recording started.
INFO:aiy.assistant.grpc:You said: "play".
INFO:aiy.assistant.grpc:You said: "play cat".
INFO:aiy.assistant.grpc:You said: "play".
INFO:aiy.assistant.grpc:You said: "Blake to".
INFO:aiy.assistant.grpc:You said: "Blake tooth".
INFO:aiy.assistant.grpc:You said: "play toots".
INFO:aiy.assistant.grpc:You said: "play Toto".
INFO:aiy.assistant.grpc:You said: "Blake to tie".
INFO:aiy.assistant.grpc:You said: "Blake to time".
INFO:aiy.assistant.grpc:You said: "blink 2 times".
INFO:aiy.assistant.grpc:End of audio request detected.


Bye
